In [1]:
import time
import sys
import os
# reload(sys)
# sys.setdefaultencoding('utf8')
import numpy as np
import pandas as pd
import datetime
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.utils.data as data_
import torch
import argparse
from torch.utils.data import IterableDataset
from datetime import datetime
import itertools
import io
from random import shuffle
import torch.nn.functional as F
import random
import math
import pickle
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
import torch.nn.utils.rnn as rnn_utils
from torch.nn import Linear, ReLU, Sigmoid, Module, BCELoss
from torch.optim.lr_scheduler import ReduceLROnPlateau,CosineAnnealingLR,StepLR
from torch.nn.utils import weight_norm
from math import sqrt
import torch.nn.functional as F
import hashlib

In [2]:
import pickle


with open('./data_bins/bin_edges_dict_dz_numeric_fusing_v5_8bin.pkl', 'rb') as handle:
    bin_edges_dict_dz_numeric = pickle.load(handle)

# print(bin_edges_dict_dz_numeric)
    
with open('./data_bins/bin_edges_dict_inquery_numeric_fusing_v5_8bin.pkl', 'rb') as handle:
    bin_edges_dict_inquery_numeric = pickle.load(handle)
print(bin_edges_dict_inquery_numeric)
   
    
with open('./data_bins/bin_edges_dict_creditos_numeric_fusing_v5_8bin.pkl', 'rb') as handle:
    bin_edges_dict_creditos_numeric = pickle.load(handle)    
# print(bin_edges_dict_creditos_numeric)

    
with open('./data_bins/bin_edges_dict_person_numeric_fusing_v5_8bin.pkl', 'rb') as handle:
    bin_edges_dict_person_numeric = pickle.load(handle)

{'inquiry_amt': Index([1000.0, 250000000.0], dtype='float64'), 'inquiry_date_ago': Index([58.0, 119.0, 190.0, 272.0, 365.0, 468.0, 588.0, 730.0], dtype='float64')}


In [3]:
class Tokenizer:
    def __init__(self, token_to_index=None, index_to_token=None):
        self.token_to_index = token_to_index or {}
        self.index_to_token = index_to_token or {}
        self.unk_token = "<unk>"
        self.unk_index = 0  # 默认将<unk>映射到0

    def fit(self, data):
        # 从数据中构建token_to_index和index_to_token字典
        unique_tokens = list(set(data))
        self.token_to_index = {token: index for index, token in enumerate(unique_tokens, start=1)}
        self.index_to_token = {index: token for token, index in self.token_to_index.items()}
        self.unk_index = 0  # 重置<unk>的索引

    def transform(self, data):
        # 编码数据为索引，未知特征使用<unk>的索引
        return [self.token_to_index.get(item, self.unk_index) for item in data]


    def reverse_transform(self, data):
        # 解码数据为特征，将0映射回<unk>
        return self.index_to_token[index] if data != self.unk_index else self.unk_token


responsibility_list = [
    'I', 'O', 'T', 'M', 'A'
]
responsibility_list_len =  len(responsibility_list)
responsibility_token_to_index = {value: index + 1 for index, value in enumerate(responsibility_list)}
responsibility_index_to_token = {index + 1: value for index, value in enumerate(responsibility_list)}
responsibility_tokenizer = Tokenizer(responsibility_token_to_index,responsibility_index_to_token)


lender_list = [
    'SIC', 'BANCO / BC', 'FINANCIERA / BC', 'DIDI PAY', 'COMPANIA DE PRESTAMO PERSONAL', 
    'SOCIEDAD FINANCIERA DE OBJETO MULTIPLE', 'MICROFINANCIERA', 'BANCOS', 
    'CONSUMIDOR FINAL / BC', 'TIENDA COMERCIAL / BC', 'GUBERNAMENTALES / BC', 'COMUNICACIONES / BC', 
    'MICROFINANCIERA / BC', 'FONDOS Y FIDEIC / BC', 'SERVICIOS / BC', 'BANCO', 
    'SOCIEDADES FINANCIERAS POPULARES', 'TELEFONIA CELULAR', 'SERVICIOS', 'FINANCIERA', 
    'AUTOMOTRIZ / BC', 'REPORTE DE CREDITO ESPECIAL', 'CIA Q  OTORGA / BC', 'TIENDA DEPARTAMENTAL', 
    'SERVS. GRALES. / BC', 'CONSUMIDOR FINAL', 'VENTA POR CATALOGO', 'GUBERNAMENTALES', 
    'COOPERATIVA / BC', 'TIENDA COMERCIAL', 'BIENES RAICES / BC', 'COMUNICACIONES', 
    'AUTOMOTRIZ', 'SIC / BC', 'TIENDA DE ROPA', 'FONDOS Y FIDEIC', 'CAJAS DE AHORRO', 
    'TIENDA DE AUTOSERVICIO', 'ARRENDAMIENTO / BC', 'ARRENDADORAS NO FINANCIERAS', 
    'COMPANIA DE FINANCIAMIENTO AUTOMOTRIZ', 'AUTOFINANCIAMIENTO', 'REPORTE ESPECIAL / BC', 
    'MONITOREO DE CREDITO', 'SOFOM-ENR / BC', 'CIA Q  OTORGA', 'SERVS. GRALES.', 
    'COMPANIA DE FINANCIAMIENTO DE MOTOCICLET', 'MERCANCIA PARA HOGAR Y OFICINA', 
    'COOPERATIVA DE AHORRO Y CREDITO', 'HIPOTECARIA / BC', 'MERCANCIA PARA LA CONSTRUCCION', 
    'OTROS VEHICULOS / BC', 'CASA DE EMPENO', 'FACTORAJE', 'BIENES RAICES', 'FIANZAS / BC', 
    'null', 'INSUMOS', 'EDUCACION / BC', 'COOPERATIVA', 'SEGURO / BC', 'SOFOL PRESTAMO PERSONAL', 
    'FONDOS Y FIDEICOMISOS', 'UNION DE CREDITO / BC', 'SERVICIO MEDICO / BC', 'HIPOTECARIO NO BANCARIO', 
    'REPORTE ESPECIAL', 'ARRENDAMIENTO', 'AGROPECUARIO / BC', 'GOBIERNO', 'ARRENDADORAS FINANCIERAS', 
    'UNION DE CREDITO', 'SOFOM-ENR', 'CIRCULO DE CREDI / BC', 'EDITORIAL / BC', 'SALUD Y SERVICIOS MEDICOS', 
    'HIPOTECARIA', 'SERVICIO DE TELEVISION DE PAGA', 'COBRANZA / BC', 'ADMINISTRADORAS DE CARTERA', 'EDUCACION', 
    'SOFOM-ER / BC', 'SEGUROS Y FIANZAS', 'OTROS VEHICULOS', 'FIANZAS', 'FACTORAJE / BC', 'SEGURO', 
    'COMP PETROLERA / BC', 'SERVICIO MEDICO', 'SOCIEDAD FINANCIERA COMUNITARIA', 'BANCOS EXTR / BC', 
    'CRED BANCO PFAE / BC', 'EDITORIAL', 'COMPANIA AGROPECUARIA', 'COBRANZA', 'SOFOM-ER', 
    'TELEFONIA LOCAL Y DE LARGA DISTANCIA', 'BANCOS EXTR', 'AGROPECUARIO'
]
lender_list_len = 100
lender_token_to_index = {value: index + 1 for index, value in enumerate(lender_list)}
lender_index_to_token = {index + 1: value for index, value in enumerate(lender_list)}
lender_tokenizer = Tokenizer(lender_token_to_index,lender_index_to_token)



inquiry_type_list = [
    'F', 'TC', 'NC', 'M', 'PP', 'R', 'Q', 'LC', 'CA', 'OT', 'AM', 'PG', 'E', 'PN', 'AE', 'P', 'PM', 'AR', 
    'NG', 'A', 'FI', 'L', 'null', 'TG', 'FT', 'LR', 'CO', 'CF', 'EQ', 'VR', 'ED', 'AV', 'MC', 'BL', 'SH', 'HE'
]
inquiry_type_list_len = 36
inquiry_type_token_to_index = {value: index + 1 for index, value in enumerate(inquiry_type_list)}
inquiry_type_index_to_token = {index + 1: value for index, value in enumerate(inquiry_type_list)}
inquiry_type_tokenizer = Tokenizer(inquiry_type_token_to_index,inquiry_type_index_to_token)


account_type_list = [
   'F', 'R', 'L', 'H', 'P', 'Q', 'A', 'E'
]
account_type_list_len =  len(account_type_list)
account_type_token_to_index = {value: index + 1 for index, value in enumerate(account_type_list)}
account_type_index_to_token = {index + 1: value for index, value in enumerate(account_type_list)}
account_type_tokenizer = Tokenizer(account_type_token_to_index,account_type_index_to_token)


credit_type_list = [
    'PP', 'CC', 'TC', 'LC', 'PN', 'AM', 'OT', 'CP', 'GS', 'HV', 'LR', 'null', 'CF', 'CA', 'BR', 'AR', 'PG', 'MC',
    'BC', 'AE', 'PM', 'AA', 'NG', 'PQ', 'PB', 'TD', 'PE', 'FT', 'FI', 'CO', 'EQ', 'NC', 'AB', 'FF', 'TS', 'HE', 'TG',
    'SH', 'AV', 'ND', 'PC', 'VR', 'ED'
]
credit_type_list_len =  len(credit_type_list)
credit_type_token_to_index = {value: index + 1 for index, value in enumerate(credit_type_list)}
credit_type_index_to_token = {index + 1: value for index, value in enumerate(credit_type_list)}
credit_type_tokenizer = Tokenizer(credit_type_token_to_index,credit_type_index_to_token)


pay_freq_list = [
    'M', 'S', 'Q', 'R', 'U', 'C', 'B', 'T', 'D', 'A', 'E'
]
pay_freq_list_len =  len(pay_freq_list)
pay_freq_token_to_index = {value: index + 1 for index, value in enumerate(pay_freq_list)}
pay_freq_index_to_token = {index + 1: value for index, value in enumerate(pay_freq_list)}
pay_freq_tokenizer = Tokenizer(pay_freq_token_to_index,pay_freq_index_to_token)


risk_comment_list = [
    "null", "CC", "UP", "PC", "CL", "CV", "IA", "LS", "CA", "LC",
    "RV", "LG", "LO", "CD", "NV", "PR", "RA", "SG", "GP", "FR",
    "VR", "IS", "NA", "RI", "FN", "AD", "PD", "IM", "FD", "PE",
    "PI", "RN", "MP", "ST", "CO", "FP", "RF", "EL", "TC", "IR"
]
risk_comment_list_len =  len(risk_comment_list)
risk_comment_token_to_index = {value: index + 1 for index, value in enumerate(risk_comment_list)}
risk_comment_index_to_token = {index + 1: value for index, value in enumerate(risk_comment_list)}
risk_comment_tokenizer = Tokenizer(risk_comment_token_to_index,risk_comment_index_to_token)


pay_status_list = [
    ' V', '13', '01', '06', '02', '84', '--', '03', '04', '05', '12', 
    '08', '11', '07', '10', '09', '27', '14', '53', '15', 
    '16', '20', '17', '18', '19', '24', '21', '22', '23', '26', 
    '25', '47', '28', '29', '30', '31', '41', '32', '36', '40', 
    '38', '42', '33', '55', '43', '39', '52', '59', '34', '46', 
    '54', '37', '35', '60', '58', '45', '48', '44', '50', '56', 
    '51', '49', '57', '61', '62', '64', '66', '67', '68', '70', 
    '69', '65', '63', '71', '72', '73', '74', '78', '77', '75', 
    '76', '80', '79', '83', '81', '82'
]
pay_status_list_len = len(pay_status_list)
pay_status_token_to_index = {value: index + 1 for index, value in enumerate(pay_status_list)}
pay_status_index_to_token = {index + 1: value for index, value in enumerate(pay_status_list)}
pay_status_tokenizer = Tokenizer(pay_status_token_to_index,pay_status_index_to_token)

 
person_fe_categorical = ['gender','education','month_income']
gender_list = [
    '-1', '0','1','2'
]
gender_list_len = len(gender_list)
gender_token_to_index = {value: index + 1 for index, value in enumerate(gender_list)}
gender_index_to_token = {index + 1: value for index, value in enumerate(gender_list)}
gender_tokenizer = Tokenizer(gender_token_to_index,gender_index_to_token)


education_list = [
    '-1', '0','1','2','3','4','5','6'
]
education_list_len = len(education_list)
education_token_to_index = {value: index + 1 for index, value in enumerate(education_list)}
education_index_to_token = {index + 1: value for index, value in enumerate(education_list)}
education_tokenizer = Tokenizer(education_token_to_index,education_index_to_token)


month_income_list =  [
    '5,000-10,000MXN', '10,000-20,000MXN','3,000-5,000MXN','20,000-50,000MXN','1,000-3,000MXN','below 1,000MXN','above 50,000MXN'
]
month_income_list_len = len(month_income_list)
month_income_token_to_index = {value: index + 1 for index, value in enumerate(month_income_list)}
month_income_index_to_token = {index + 1: value for index, value in enumerate(month_income_list)}
month_income_tokenizer = Tokenizer(month_income_token_to_index,month_income_index_to_token)




def map_values_to_bins(values_str, bin_edges):
    values = np.array(list(map(float, values_str.split(','))))  # 将字符串值转换成浮点数列表
    binned_values =  np.digitize(values, bins=bin_edges, right=True).tolist()
    binned_values = [int(x) if not (x == float('inf') or x == float('-inf')) else 0 for x in binned_values]

    return binned_values


def map_values_to_bins_single(values_str, bin_edges):
    values = np.array(list(map(float, values_str.split(','))))  # 将字符串值转换成浮点数列表
    binned_values =  np.digitize(values, bins=bin_edges, right=True).tolist()[0]
#     print(binned_values)
    return binned_values


class MyData(IterableDataset):
    def __init__(self, file_paths, is_train=True, augmentation_prob=0.8,shuffle_files=True):
        if shuffle_files:
            shuffle(file_paths)
        self.file_paths = file_paths
        self.is_train = is_train
        self.augmentation_prob = augmentation_prob
        
    def __iter__(self):
        worker_info = torch.utils.data.get_worker_info()
        world_size = 1
        rank = 0

        worker_id = 0
        mod = world_size
        shift = rank

        if worker_info:
            num_workers = worker_info.num_workers
            worker_id = worker_info.id
            mod *= num_workers
            shift = rank * num_workers + worker_id


        line_iter = itertools.chain(*[io.open(f, encoding="utf-8") for f in self.file_paths])
        for i, line in enumerate(line_iter):
            if (i + shift) % mod == 0:
                return_line = self.process(line)
                if return_line:
                    yield return_line


    def process(self, line):
        text_data = line
        dz_categorica_feature, dz_numeric_feature, person_feature , label, \
        len_dz, cfrnid, bv1_score,date_time_credit, inquery_feature, creditos_feature, len_inquery, len_creditos = process_one_sample(text_data)
        dz_categorica_feature = torch.tensor(dz_categorica_feature)
        dz_numeric_feature = torch.tensor(dz_numeric_feature)
        person_feature = torch.tensor(person_feature)
        label = torch.tensor(label)
        if cfrnid != 360287970202376449:
            return False
        cfrnid = torch.tensor(cfrnid)
        bv1_score = torch.tensor(bv1_score)
        date_time_credit = torch.tensor(date_time_credit)
        
        inquery_feature = torch.tensor(inquery_feature)
        creditos_feature = torch.tensor(creditos_feature)
        
        tuple_ = (dz_categorica_feature, dz_numeric_feature, person_feature,label, len_dz, cfrnid, bv1_score,date_time_credit, inquery_feature, creditos_feature, len_inquery, len_creditos)
        return tuple_
    
    

def process_2d_list(lst, k, pad_token=120):
    result = []
    for sub_lst in lst:
        if len(sub_lst) < k:
            # 如果子列表长度不足k，补充0
            sub_lst += [pad_token] * (k - len(sub_lst))
            result.append(sub_lst)
        else:
            # 如果子列表长度超过k，截取前k个元素
            result.append(sub_lst[-k:])
    return result


In [48]:
def process_one_sample(features,max_length=120):
    [cfrnid ,slice_dt ,uid,if_zaidai,bcard_fuse_v7_model_score,bv6_jingying,zx_v5,credit_card_cfrnid,clear_tag,mob_tag,everoverdue_tag,channel_tag,fixed_credit_limit_tag,risk_unfreeze_d7_tag,t30_trade_dob90_dpd7_fz,t30_trade_dob90_dpd7_fm,all_dob45_dpd7_fz,all_dob45_dpd7_fm,all_dob90_dpd7_fz,all_dob90_dpd7_fm,all_dob90_dpd30_fz,all_dob90_dpd30_fm,all_dob120_dpd7_fz,all_dob120_dpd7_fm,all_dob120_dpd30_fz,all_dob120_dpd30_fm,all_dob180_dpd7_fz,all_dob180_dpd7_fm,all_dob180_dpd30_fz,all_dob180_dpd30_fm,dob180_order_mths,if_t30_120order, \
    gender,education,month_income,domicilios_state_distinct_nums,domicilios_street_distinct_nums,domicilios_city_distinct_nums,domicilios_colony_distinct_nums,scores_fico_score_max,domicilio_len,persona_dependent_num,persona_age, \
    is_today_credit_submit,is_credit_succ,is_today_credit_succ,limit_raised_flag,is_multiterm,is_curr_day_loan_apply,is_curr_day_loan_success,is_curr_onloan,is_curr_clear,is_curr_overdue,d30_active_flag,d60_active_flag,d90_active_flag,freeze_flag, \
    credit_submit_cnt,first_credit_submit_date,last_credit_submit_days,today_credit_submit_cnt,credit_succ_days,credit_refuse_cnt,last_credit_refuse_days,initial_credit_line,fixed_credit_imit,freeze_limit,total_used_limit,used_limit,limit_raised_times,total_raised_limit,last_raised_limit,total_rate,last_adjust_period_date,first_loan_success_day,curr_day_loan_principal,last_loan_clear_day,total_loan_apply_cnt,total_loan_success_cnt,total_loan_clean_cnt,total_loan_fail_cnt,total_loan_success_principal,total_loan_clear_principal,curr_day_plan_repay_order_cnt,curr_day_real_repay_order_cnt,curr_day_plan_repay_principal_interest,curr_day_real_repay_principal_interest,total_day_plan_repay_order_cnt,total_day_plan_repay_principal,total_day_plan_repay_interest,total_day_real_repay_order_cnt,total_day_real_repay_principal,total_day_real_repay_interest,curr_max_overdue_days,curr_overdue_order_nums,curr_overdue_principal,curr_overdue_interest,curr_overdue_balance,his_max_overdue_days,his_overdue_order_cnt,last_overdue_date,his_overdue_order_duration_cnt,last_normal_loan_date,curr_loan_order_cnt,curr_loan_order_normal_cnt,curr_loan_order_dpd2_cnt,curr_loan_order_principal,curr_loan_order_normal_principal,curr_loan_order_dpd2_principal,curr_loan_order_balance,curr_loan_order_normal_balance,curr_loan_order_dpd2_balance,interest_bearing_balance,recency,total_balance_principal_interest_add,curr_balance_principal_intetest_add,first_succ_repay_dbt,first_dur_settle_dbt,last_succ_repay_dbt,last_dur_settle_dbt,his_avg_adv_days,his_max_adv_days,his_min_adv_days,his_avg_ovd_days,his_max_ovd_days,his_min_ovd_days,his_adv_dur_num,his_ovd_dur_num,his_avg_ord_settle_day,his_max_ord_settle_day,his_min_ord_settle_day,his_repay_credit_line_rate,his_adv_repay_principal,his_adv_repay_credit_line_rate,his_ontime_repay_principal,his_ontime_repay_credit_line_rate,his_ovd_repay_principal,his_ovd_repay_credit_line_rate,his_settle_principal,his_max_repay_principal,his_min_repay_principal,his_avg_repay_principal,his_max_day_repay_principal,his_min_day_repay_principal,his_avg_day_repay_principal,last_1_time_repay_days_from_last_repay,last_1_month_avg_adv_days,last_1_month_max_adv_days,last_1_month_min_adv_days,last_1_month_avg_ovd_days,last_1_month_max_ovd_days,last_1_month_min_ovd_days,last_1_month_adv_dur_num,last_1_month_ovd_dur_num,last_1_month_avg_ord_settle_day,last_1_month_max_ord_settle_day,last_1_month_min_ord_settle_day,last_1_month_repay_principal,last_1_month_repay_credit_line_rate,last_1_month_adv_repay_credit_line_rate,last_1_month_ontime_repay_credit_line_rate,last_1_month_ovd_repay_principal,last_1_month_ovd_repay_credit_line_rate,last_1_month_max_repay_principal,last_1_month_min_repay_principal,last_1_month_avg_repay_principal,last_1_month_max_day_repay_principal,last_1_month_min_day_repay_principal,last_1_month_avg_day_repay_principal,last_1_month_onerepy_order_num,last_1_month_onerepy_dur_num,last_1_month_offline_repay_num,last_1_month_online_repay_num,last_1_month_offline_repay_num_rate,last_1_month_online_repay_num_rate,last_1_month_offline_repay_principal,last_1_month_online_repay_principal,last_1_month_offline_repay_principal_rate,last_1_month_online_repay_principal_rate,last_1_time_repay_amount,last_1_time_plan_amonut,last_1_time_repay_days_from_last_loan,last_1_time_repay_days_last_repay,max_total_durations,max_rest_duration_num,cash_loan_balance,last_1_time_repay_loan,ovd_prin_interest,max_ovd_days,first_ovd_days_before_today,last_ovd_days_before_today,first_dur_days_before_today,norepay_first_dur_days_before_today,his_ovd_prin,his_ovd_credit_line,his_ovd_prin_credit_rate,his_ovd_loan_principal,his_ovd_prin_loan_rate,last_1_month_order_num,last_1_month_dpd3_order_num,last_1_month_dpd3_order_rate,last_1_month_dpd7_order_num,last_1_month_dpd7_order_rate,last_1_month_dpd30_order_num,last_1_month_dpd30_order_rate,last_30_day_max_ovd_prin,last_30_day_max_ovd_credit_line,last_30_day_max_ovd_prin_credit_rate,last_1_month_fpd3_order_num,last_1_month_fpd7_order_num,last_1_month_fpd30_order_num,last_1_month_fpd3_ovd_prin,last_1_month_fpd7_ovd_prin,last_1_month_fpd30_ovd_prin,last_1_month_max_fpd,last_1_month_avg_fpd,last_1_month_min_fpd,sum_loan_fail_orders,sum_loan_success_orders,sum_loan_clear_order_num,sum_loan_clear_order_duration_num,his_disburse_succ_sum_amount,now_max,now_avg,his_max_onloan_day,his_min_onloan_day,his_avg_onloan_day,his_max_onloan_rate,his_min_onloan_rate,his_avg_onloan_rate,last_loan_freez_days,normal_onloan_order_num,last_clear_onloan_days,last_loan_total_durations,last_1_month_loan_fail_orders,last_1_month_loan_success_orders,last_1_month_loan_clear_order_num,last_1_month_loan_clear_order_duration_num,last_1_month_max_loan_to_clear_day,last_1_month_min_loan_to_clear_day,last_1_month_avg_loan_to_clear_day,last_1_month_max_loan_credit_used_rate,last_1_month_min_loan_credit_used_rate,last_1_month_avg_loan_credit_used_rate,first_credit_line,last_credit_line,sum_balance_principal,loan_credit_used_rate,last_freeze_days,his_credit_fail_num,first_success_days,last_3month_max_used_limit_rate,last_3month_max_used_limit,last_1month_max_used_limit_rate,last_1month_max_used_limit,last_2month_max_used_limit_rate,last_2month_max_used_limit,last_3month_avg_used_limit_rate,last_3month_avg_used_limit,last_3month_sum_used_limit,last_1month_avg_used_limit_rate,last_1month_avg_used_limit,last_1month_sum_used_limit,last_6month_avg_used_limit_rate,last_6month_avg_used_limit,last_6month_sum_used_limit,last_2month_avg_used_limit_rate,last_2month_avg_used_limit,last_2month_sum_used_limit,last_2_1month_avg_used_limit_rate,last_2_1month_avg_used_limit,last_2_1month_max_used_limit_rate,last_2_1month_max_used_limit,last_3_1month_avg_used_limit_rate,last_3_1month_avg_used_limit,last_3_1month_max_used_limit_rate,last_3_1month_max_used_limit,use_proportion_up,use_proportion_down,use_proportion_all,last_3_1month_proportion_of_quota,last_2_1month_proportion_of_quota,last_1_1month_proportion_of_quota,last_3_1month_proportion_of_quota_max,last_2_1month_proportion_of_quota_max,last_1_1month_proportion_of_quota_max,last_3_1month_dvalue_of_quota,last_2_1month_dvalue_of_quota,last_1_1month_dvalue_of_quota,last_3_1month_dvalue_of_quota_max,last_2_1month_dvalue_of_quota_max,last_1_1month_dvalue_of_quota_max,last_3month_up_from_last_month,last_3month_down_from_last_month,last_3month_unchange_from_last_month,last_3month_zero_month_num,last_3month_greater90_month_num,last_3month_greater80_month_num,last_3month_greater50_month_num,last_1_month_credit_home_num,last_1_month_credit_home_duration,last_1_month_withdraw_num,last_1_month_withdraw_duration,last_1_month_max_credit_duration,last_1_month_avg_credit_duration,last_1_month_min_credit_duration,last_1_month_total_message_num,last_1_month_click_message_num,last_1_month_total_resource_num,last_1_month_click_resource_num,last_1_month_total_email_num,last_1_month_click_email_num, \
    inquiry_responsibility, inquiry_type, inquiry_lender, inquiry_amt, inquiry_date_ago, total_inquery_times, \
    lender, responsibility, account_type, credit_type, pay_freq, is_account_close, pay_status, risk_comment, has_garantee_insurance, pay_no, pay_amt, \
    open_date_ago, last_pay_date_ago, last_buy_date_ago, account_live_days, last_report_date_ago, max_redit_amt, balance, limit, overdue_amt, \
    overdue_no, length_pay_status_his, max_overdue_amt, max_overdue_date_ago, last_overdue_amt, max_overdue_num, overdue_num_ratio, \
    max_credit_limit_rate, max_bal_rate, max_bal_rate2, total_creditos_times]  = features
    
    total_creditos_times = total_creditos_times.strip('\n')
    
    cfrnid = int(cfrnid)
    date_time_credit = int(slice_dt.replace('-', ''))
    label = [int(all_dob45_dpd7_fz),int(all_dob90_dpd7_fz),int(all_dob90_dpd30_fz), \
             int(all_dob120_dpd7_fm),int(all_dob120_dpd7_fz),int(all_dob120_dpd30_fm),int(all_dob120_dpd30_fz) \
            ,int(all_dob180_dpd7_fm),int(all_dob180_dpd7_fz),int(all_dob180_dpd30_fm),int(all_dob180_dpd30_fz)]
    
    bv1_score = float(bv6_jingying)
    
    cls_token = 121
    
    # process person
    gender = gender_tokenizer.transform([gender])[0]
    education = education_tokenizer.transform([education])[0]
    month_income = month_income_tokenizer.transform([month_income])[0]
    person_categorica_feature = [gender,education,month_income]
    person_numeric_feature = [domicilios_state_distinct_nums,domicilios_street_distinct_nums,domicilios_city_distinct_nums,domicilios_colony_distinct_nums,scores_fico_score_max,domicilio_len,persona_dependent_num,persona_age]
    person_numeric_feature_discretized = [map_values_to_bins_single(data_str, bin_edges_dict_person_numeric[feature_name]) for feature_name, data_str in zip(list(bin_edges_dict_person_numeric.keys()), person_numeric_feature)]

    person_feature = person_categorica_feature  + person_numeric_feature_discretized


    
    
    dz_categorica_feature = [limit_raised_flag , is_multiterm ,is_curr_onloan , is_curr_clear]
    dz_categorica_feature = [fe.split(',') for fe in dz_categorica_feature]
    dz_numeric_feature = [credit_submit_cnt,first_credit_submit_date,last_credit_submit_days,today_credit_submit_cnt,credit_succ_days,credit_refuse_cnt,last_credit_refuse_days,initial_credit_line,fixed_credit_imit,freeze_limit,total_used_limit,used_limit,limit_raised_times,total_rate,last_adjust_period_date,first_loan_success_day,curr_day_loan_principal,last_loan_clear_day,total_loan_apply_cnt,total_loan_success_cnt,total_loan_clean_cnt,total_loan_fail_cnt,total_loan_success_principal,total_loan_clear_principal,curr_day_plan_repay_order_cnt,curr_day_real_repay_order_cnt,curr_day_plan_repay_principal_interest,curr_day_real_repay_principal_interest,total_day_plan_repay_order_cnt,total_day_plan_repay_principal,total_day_plan_repay_interest,total_day_real_repay_order_cnt,total_day_real_repay_principal,total_day_real_repay_interest,curr_max_overdue_days,curr_overdue_order_nums,curr_overdue_principal,curr_overdue_interest,curr_overdue_balance,his_max_overdue_days,his_overdue_order_cnt,last_overdue_date,his_overdue_order_duration_cnt,last_normal_loan_date,curr_loan_order_cnt,curr_loan_order_normal_cnt,curr_loan_order_dpd2_cnt,curr_loan_order_principal,curr_loan_order_normal_principal,curr_loan_order_dpd2_principal,curr_loan_order_balance,curr_loan_order_normal_balance,curr_loan_order_dpd2_balance,interest_bearing_balance,recency,total_balance_principal_interest_add,curr_balance_principal_intetest_add,first_succ_repay_dbt,first_dur_settle_dbt,last_succ_repay_dbt,last_dur_settle_dbt,his_avg_adv_days,his_max_adv_days,his_min_adv_days,his_avg_ovd_days,his_max_ovd_days,his_min_ovd_days,his_adv_dur_num,his_ovd_dur_num,his_avg_ord_settle_day,his_max_ord_settle_day,his_min_ord_settle_day,his_repay_credit_line_rate,his_adv_repay_principal,his_adv_repay_credit_line_rate,his_ontime_repay_principal,his_ontime_repay_credit_line_rate,his_ovd_repay_principal,his_ovd_repay_credit_line_rate,his_settle_principal,his_max_repay_principal,his_min_repay_principal,his_avg_repay_principal,his_max_day_repay_principal,his_min_day_repay_principal,his_avg_day_repay_principal,last_1_time_repay_days_from_last_repay,last_1_month_avg_adv_days,last_1_month_max_adv_days,last_1_month_min_adv_days,last_1_month_avg_ovd_days,last_1_month_max_ovd_days,last_1_month_min_ovd_days,last_1_month_adv_dur_num,last_1_month_ovd_dur_num,last_1_month_avg_ord_settle_day,last_1_month_max_ord_settle_day,last_1_month_min_ord_settle_day,last_1_month_repay_principal,last_1_month_repay_credit_line_rate,last_1_month_adv_repay_credit_line_rate,last_1_month_ontime_repay_credit_line_rate,last_1_month_ovd_repay_principal,last_1_month_ovd_repay_credit_line_rate,last_1_month_max_repay_principal,last_1_month_min_repay_principal,last_1_month_avg_repay_principal,last_1_month_max_day_repay_principal,last_1_month_min_day_repay_principal,last_1_month_avg_day_repay_principal,last_1_month_onerepy_order_num,last_1_month_onerepy_dur_num,last_1_month_offline_repay_num,last_1_month_online_repay_num,last_1_month_offline_repay_num_rate,last_1_month_online_repay_num_rate,last_1_month_offline_repay_principal,last_1_month_online_repay_principal,last_1_month_offline_repay_principal_rate,last_1_month_online_repay_principal_rate,last_1_time_repay_amount,last_1_time_plan_amonut,last_1_time_repay_days_from_last_loan,last_1_time_repay_days_last_repay,ovd_prin_interest,max_ovd_days,first_ovd_days_before_today,last_ovd_days_before_today,first_dur_days_before_today,norepay_first_dur_days_before_today,his_ovd_prin,his_ovd_credit_line,his_ovd_prin_credit_rate,his_ovd_loan_principal,his_ovd_prin_loan_rate,last_1_month_order_num,last_30_day_max_ovd_prin,last_30_day_max_ovd_credit_line,last_30_day_max_ovd_prin_credit_rate,last_1_month_max_fpd,last_1_month_avg_fpd,last_1_month_min_fpd,sum_loan_fail_orders,sum_loan_success_orders,sum_loan_clear_order_num,sum_loan_clear_order_duration_num,his_disburse_succ_sum_amount,now_max,now_avg,his_max_onloan_day,his_min_onloan_day,his_avg_onloan_day,his_max_onloan_rate,his_min_onloan_rate,his_avg_onloan_rate,last_loan_freez_days,normal_onloan_order_num,last_clear_onloan_days,last_loan_total_durations,last_1_month_loan_fail_orders,last_1_month_loan_success_orders,last_1_month_loan_clear_order_num,last_1_month_loan_clear_order_duration_num,last_1_month_max_loan_to_clear_day,last_1_month_min_loan_to_clear_day,last_1_month_avg_loan_to_clear_day,last_1_month_max_loan_credit_used_rate,last_1_month_min_loan_credit_used_rate,last_1_month_avg_loan_credit_used_rate,first_credit_line,last_credit_line,sum_balance_principal,loan_credit_used_rate,last_freeze_days,his_credit_fail_num,first_success_days,last_3month_max_used_limit_rate,last_3month_max_used_limit,last_1month_max_used_limit_rate,last_1month_max_used_limit,last_2month_max_used_limit_rate,last_2month_max_used_limit,last_3month_avg_used_limit_rate,last_3month_avg_used_limit,last_3month_sum_used_limit,last_1month_avg_used_limit_rate,last_1month_avg_used_limit,last_1month_sum_used_limit,last_6month_avg_used_limit_rate,last_6month_avg_used_limit,last_6month_sum_used_limit,last_2month_avg_used_limit_rate,last_2month_avg_used_limit,last_2month_sum_used_limit,last_2_1month_avg_used_limit_rate,last_2_1month_avg_used_limit,last_2_1month_max_used_limit_rate,last_2_1month_max_used_limit,last_3_1month_avg_used_limit_rate,last_3_1month_avg_used_limit,last_3_1month_max_used_limit_rate,last_3_1month_max_used_limit,use_proportion_up,use_proportion_down,use_proportion_all,last_3_1month_proportion_of_quota,last_2_1month_proportion_of_quota,last_1_1month_proportion_of_quota,last_3_1month_proportion_of_quota_max,last_2_1month_proportion_of_quota_max,last_1_1month_proportion_of_quota_max,last_3_1month_dvalue_of_quota,last_2_1month_dvalue_of_quota,last_1_1month_dvalue_of_quota,last_3_1month_dvalue_of_quota_max,last_2_1month_dvalue_of_quota_max,last_1_1month_dvalue_of_quota_max,last_3month_up_from_last_month,last_3month_down_from_last_month,last_3month_unchange_from_last_month,last_3month_zero_month_num,last_3month_greater90_month_num,last_3month_greater80_month_num,last_3month_greater50_month_num,last_1_month_credit_home_num,last_1_month_credit_home_duration,last_1_month_withdraw_num,last_1_month_withdraw_duration,last_1_month_max_credit_duration,last_1_month_avg_credit_duration,last_1_month_min_credit_duration,last_1_month_total_message_num,last_1_month_click_message_num,last_1_month_total_resource_num,last_1_month_click_resource_num,last_1_month_total_email_num,last_1_month_click_email_num]
    dz_numeric_feature = [map_values_to_bins(data_str, bin_edges_dict_dz_numeric[feature_name]) for feature_name, data_str in zip(list(bin_edges_dict_dz_numeric.keys()), dz_numeric_feature)]   
    max_length = 25
    len_dz= len(dz_categorica_feature[0])
    len_dz = len_dz if len_dz <=max_length else max_length
    dz_categorica_feature = [[int(value) for value in row] for row in dz_categorica_feature]
    dz_numeric_feature = [[float(value) if value != '' else 0.0 for value in row] for row in dz_numeric_feature]
    dz_numeric_feature = [row[::-1] for row in dz_numeric_feature]
    dz_categorica_feature = process_2d_list(dz_categorica_feature,max_length)
    dz_categorica_feature = [[cls_token] + sublist for sublist in dz_categorica_feature]
    dz_numeric_feature = process_2d_list(dz_numeric_feature,max_length)
    dz_numeric_feature = [[cls_token] + sublist for sublist in dz_numeric_feature]


    # process inquery - total_inquery_times
    inquery_max_length = 119
    inquery_numeric_feature = [inquiry_amt, inquiry_date_ago]
    inquery_numeric_feature = [map_values_to_bins(data_str, bin_edges_dict_inquery_numeric[feature_name]) for feature_name, data_str in zip(list(bin_edges_dict_inquery_numeric.keys()), inquery_numeric_feature)]
    
    inquery_cate = [inquiry_responsibility,inquiry_lender,inquiry_type]
    inquery_cate = [fe.split(',') for fe in inquery_cate]
    len_inquery = len(inquery_cate[0])
    len_inquery = len_inquery if len_inquery <=inquery_max_length else inquery_max_length
    inquiry_responsibility = responsibility_tokenizer.transform(inquery_cate[0])
    inquiry_lender = lender_tokenizer.transform(inquery_cate[1])
    inquiry_type = inquiry_type_tokenizer.transform(inquery_cate[2])
    inquery_feature = [inquiry_responsibility,inquiry_lender,inquiry_type] + inquery_numeric_feature
    inquery_feature = process_2d_list(inquery_feature,inquery_max_length)
    inquery_feature = [[cls_token] + sublist for sublist in inquery_feature]
    
    
    
    # process creditos  -total_creditos_times
    creditos_max_length = 199
    creditos_numeric_feature =  [pay_no, pay_amt, open_date_ago, last_pay_date_ago, last_buy_date_ago, account_live_days, last_report_date_ago, max_redit_amt, balance, limit, overdue_amt, \
    overdue_no, length_pay_status_his, max_overdue_amt, max_overdue_date_ago, last_overdue_amt, max_overdue_num, overdue_num_ratio, max_credit_limit_rate, max_bal_rate, max_bal_rate2]
    creditos_numeric_feature = [map_values_to_bins(data_str, bin_edges_dict_creditos_numeric[feature_name]) for feature_name, data_str in zip(list(bin_edges_dict_creditos_numeric.keys()), creditos_numeric_feature)]

    # - has_garantee_insurance
    creditos_cate = [lender, responsibility, account_type, credit_type, pay_freq, is_account_close, pay_status, risk_comment]        
    creditos_cate = [fe.split(',') for fe in creditos_cate]
    len_creditos = len(creditos_cate[0])
    len_creditos = len_creditos if len_creditos <=creditos_max_length else creditos_max_length
    creditos_cate[0] = lender_tokenizer.transform(creditos_cate[0])
    creditos_cate[1] = responsibility_tokenizer.transform(creditos_cate[1])
    creditos_cate[2]  = account_type_tokenizer.transform(creditos_cate[2])
    creditos_cate[3] = credit_type_tokenizer.transform(creditos_cate[3])
    creditos_cate[4] = pay_freq_tokenizer.transform(creditos_cate[4])
    creditos_cate[6] = pay_status_tokenizer.transform(creditos_cate[6])
    creditos_cate[7] = risk_comment_tokenizer.transform(creditos_cate[7])
    creditos_cate = [[int(value) for value in row] for row in creditos_cate]
    creditos_feature = creditos_cate  + creditos_numeric_feature
    creditos_feature = process_2d_list(creditos_feature,creditos_max_length)
    creditos_feature = [[cls_token] + sublist for sublist in creditos_feature]
    
    
    print('dz_categorica_feature summary include ', len(dz_categorica_feature), 'sentences',  ' and dz_categorica_feature one sentence like : ', dz_categorica_feature[1])
    print('dz_numeric_feature summary include : ', len(dz_numeric_feature), 'sentences', ' and dz_numeric_feature one sentence like : ', dz_numeric_feature[3])
    print('inquery_feature summary include : ', len(inquery_feature), 'sentences', ' and inquery_feature one sentence like : ', inquery_feature[4])
    print('creditos_feature summary include : ', len(creditos_feature), 'sentences', ' and creditos_feature one sentence like : ', creditos_feature[2])
    return dz_categorica_feature, dz_numeric_feature, person_feature, label, len_dz, cfrnid, bv1_score,date_time_credit, inquery_feature, creditos_feature, len_inquery, len_creditos


In [49]:
import json
import torch  # Assume that PyTorch is used for model evaluation

def load_data_from_json(file_path):
    """
    Load and return data from a JSON file.

    Parameters:
    - file_path (str): The path to the JSON file.

    Returns:
    - dict: The data loaded from the JSON file as a dictionary.
    """
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def evaluate(input_data):
    """
    Evaluate the model with features extracted from the input data.

    Parameters:
    - input_data (dict): The dictionary containing features and data to evaluate.

    Raises:
    - Exception: Throws an exception if necessary features are missing, or if there's an error during preprocessing or model prediction.

    Notes:
    - Assume process_one_sample and other functions are defined elsewhere and handle specific data preparation and model evaluation tasks.
    """
    try:
        data_input = []
        features_list = columns_num  # Assume columns_num is defined elsewhere and contains the list of required features
        for feature in features_list:
            data_input.append(input_data[feature])
    except Exception as e:
        raise Exception("Incomplete features for model input, please check your configuration!") from e

    try:
        # Example function call - replace with actual preprocessing and evaluation
        dz_categorica_feature, dz_numeric_feature, person_feature, label, len_dz, cfrnid, bv1_score, date_time_credit, inquery_feature, creditos_feature, len_inquery, len_creditos = process_one_sample(data_input)
        
    except Exception as e:
        raise Exception("Error during feature preprocessing!") from e
    
    try:
        val_preds = []
        val_score = []
        # Model evaluation logic goes here
        with torch.no_grad():
            pass  # Replace with actual evaluation logic
        
    except Exception as e:
        raise Exception("Anomaly during model prediction process!") from e
        
file_path = 'path_to_your_file.json'  # Specify the path to your JSON file
file_path = './data.json'
# Load data from JSON file
data_dict = load_data_from_json(file_path)

# Evaluate the model with loaded data
evaluate(data_dict)

dz_categorica_feature summary include  4 sentences  and dz_categorica_feature one sentence like :  [121, 0, 0, 1, 1, 1, 1, 1, 1, 1, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120]
dz_numeric_feature summary include :  237 sentences  and dz_numeric_feature one sentence like :  [121, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120]
inquery_feature summary include :  5 sentences  and inquery_feature one sentence like :  [121, 7, 7, 7, 6, 6, 5, 4, 4, 3, 3, 2, 2, 2, 1, 1, 1, 0, 0, 0, 0, 0, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 12